# Business Problem

Expat life in Amsterdam can be exciting but it doesn't come without challenges. One of the most common challenges for expats arriving to Amsterdam is housing. If you're a local you might have a good feeling for the neighbourhoods in Amsterdam - what unites them and how they differ. But as an expat, you lack that knowledge and it will take you quite some time to get acquainted with Amsterdam to know which neighbourhood might suit you.  

So what to do? How to determine which neighbourhood is the right one for you when you are new to Amsterdam? You could spend days of your life googling about each and every neighbourhood or you could have a look at the Amsterdam Neighbourhood Map I will be building.

# 2. Data & 3. Methodology

In order to create the Amsterdam Neighbourhood Map, we will need to cluster Amsterdam neighbourhoods based on various data points that we can obtain about the neighbourhoods. The neighbourhoods will be clustered based on data coming from the city of Amsterdam, the TomTom Search API and the Foursquare Places API.

###### ! PLEASE NOTE ! 
###### Data & Methodology have been combined into one section as most analysis was done on the data as soon as the data was imported from different sources. This was done for efficiency reasons to avoid first dumping all data into the dataframe and then making all the calculations with a huge dataset. Instead, data was imported from the different sources and right away cleaned, explored and analysed.

#### Import all needed libraries

In [555]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
import numpy as np
import requests
from geopy.geocoders import Nominatim
import geocoder # to geocode the neighbourhoods
import re # to use regular expressions
import time # 
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Amsterdam urban development data by neighbourhood

The city of Amsterdam provides a variety of datasets to the public. We will be leveraging on a variety of datasets that will give us insights into the housing stock, housing prices and residents of each neighbourhood.

#### Import the eight datasets from the city of Amsterdam

In [556]:
urls = []

for x in range(1, 9):
    urls.append('https://api.data.amsterdam.nl/dcatd/datasets/B4EcyyT9e_AFyQ/purls/{}'.format(x))

print('The following datasets have been imported:\n')

df_list = []

for url in urls:
    df = pd.read_excel(url, header=None)
    dataset_name = df[0][0]
    column_names = ['wijk/std']
    header_index = df[df[0].str.match('wijk/std') == True].index[0]

    for column in range(1, df.shape[1]):
        column_names.append(df.iloc[0:header_index+1, column].astype('str').str.cat(sep=' ') + '_' + df[0][0][0:4])
        column_names = [column.replace('nan', '').strip() for column in column_names]
            
    df.columns = column_names
    df.dropna(subset=[column_names[0]], inplace=True)
    df = df[df[column_names[0]].str.match(r'^[A-Z][0-9][0-9]')]
    df['wijk/std'] = df['wijk/std'].str.split(n=1, expand=True)    
    df.reset_index(drop=True, inplace=True)
    df_list.append(df)
    print(dataset_name, '[{} records, {} columns]'.format(df.shape[0], df.shape[1]))

df = df_list[0]

for x in range(1, len(df_list)): 
    df = df.merge(df_list[x], how='outer', on='wijk/std')

print('\n')
print('The {} datasets have been merged into one dataframe with {} records and {} columns.'.format(len(df_list), df.shape[0], df.shape[1]))

The following datasets have been imported:

7.1a  Woningvoorraad wijken en stadsdelen, 1 januari 2016-2020 [99 records, 7 columns]
7.2a   Woningvoorraad wijken en stadsdelen naar aantal kamers per woning, 1 januari 2020 [99 records, 10 columns]
7.3a  Woningvoorraad wijken en stadsdelen naar gemiddeld gebruiks oppervlakte (GBO), 1 januari 2020 [99 records, 11 columns]
7.4a  Woningvoorraad wijken en stadsdelen naar bouwperiode, 1 januari 2020 [99 records, 9 columns]
7.5a  Woningvoorraad wijken en stadsdelen naar eigendom, 1 januari 2019 [99 records, 6 columns]
7.6a  Gemiddelde WOZ-waarde woningen wijken en stadsdelen (x 1 euro), 1 januari 2017-2019 [99 records, 7 columns]
7.7a  Woningvoorraad wijken en stadsdelen en de prognoses hiervan, 1 januari 2020-2050 [100 records, 7 columns]
7.8a  Hoofdbewoners wijken en stadsdelen naar bewoningstypen, 1 januari 2020 1) [99 records, 11 columns]


The 8 datasets have been merged into one dataframe with 100 records and 61 columns.


#### Clean dataframe

In [557]:
df.rename(columns={'wijk/std':'neighbourhood_code', 'naam wijk/std_7.1a':'neighbourhood_name'}, inplace=True)
df['neighbourhood_name'] = df['neighbourhood_name'].str.replace('/', ' en ')
df['neighbourhood_name'] = df['neighbourhood_name'].str.replace(r'.[(].....[)]', '')
df.drop(columns=df.filter(regex='naam wijk/std', axis=1).columns, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = ['neighbourhood_code', 'neighbourhood_name',
                 'no_houses_2016',
                 'no_houses_2017',
                 'no_houses_2018',
                 'no_houses_2019',
                 'no_houses_2020',
                 'no_houses_2020_rooms_1+2',
                 'no_houses_2020_rooms_3',
                 'no_houses_2020_rooms_4',
                 'no_houses_2020_rooms_5+',
                 'no_houses_2020_rooms_unknown',
                 'no_houses_2020_rooms_total',
                 'perc_houses_2020_rooms_1+2',
                 'perc_houses_2020_rooms_5+',
                 'no_houses_2020_m2_<40',
                 'no_houses_2020_m2_40-50',
                 'no_houses_2020_m2_50-60',
                 'no_houses_2020_m2_60-70',
                 'no_houses_2020_m2_70-80',
                 'no_houses_2020_m2_80-90',
                 'no_houses_2020_m2_90+',
                 'no_houses_2020_m2_unknown',
                 'no_houses_2020_m2_total',
                 'no_houses_2020_year_<1946',
                 'no_houses_2020_year_1946-1980',
                 'no_houses_2020_year_1981-1990',
                 'no_houses_2020_year_1991-2000',
                 'no_houses_2020_year_2001-2010',
                 'no_houses_2020_year_2010+',
                 'no_houses_2020_year_total',
                 'no_houses_2019_owner_is_resident',
                 'no_houses_2019_corporation_rental_total',
                 'no_houses_2019_corporation_rental_free_sector',
                 'no_houses_2019_private_rental',
                 'no_houses_2019_total',
                 'avg_real_estate_value_in_eur_2017',
                 'avg_real_estate_value_in_eur_2017_per_m2',
                 'avg_real_estate_value_in_eur_2018',
                 'avg_real_estate_value_in_eur_2018_per_m2',
                 'avg_real_estate_value_in_eur_2019',
                 'avg_real_estate_value_in_eur_2019_per_m2',
                 'no_houses_forecasted_2020',
                 'no_houses_forecasted_2025',
                 'no_houses_forecasted_2030',
                 'no_houses_forecasted_2040',
                 'no_houses_forecasted_2050',
                 'no_residents_2020_couple_with_children',
                 'no_residents_2020_single_parent_family',
                 'no_residents_2020_couple_without_children',
                 'no_residents_2020_2_singles',
                 'no_residents_2020_1_single+single_parent_family',
                 'no_residents_2020_living_alone',
                 'no_residents_2020_3+_singles',
                 'no_residents_2020_other',
                 'no_residents_2020']

df.drop(columns=['no_houses_2020_rooms_total', 
                 'no_houses_2020_m2_total', 
                 'no_houses_2020_year_total', 
                 'no_houses_2019_total', 
                 'no_houses_forecasted_2020', 
                 'no_houses_2019_corporation_rental_free_sector',
                 'no_houses_2020_m2_unknown',
                 'no_houses_2020_rooms_unknown',
                 'perc_houses_2020_rooms_1+2',
                 'perc_houses_2020_rooms_5+',
                 'no_houses_2020_rooms_3',
                 'no_houses_2020_rooms_4',
                 'no_houses_2020_rooms_unknown',
                 'no_houses_2020_rooms_total',
                 'avg_real_estate_value_in_eur_2017',
                 'avg_real_estate_value_in_eur_2018',
                 'avg_real_estate_value_in_eur_2019',
                 'no_residents_2020_other'], 
        inplace=True)

df[df.columns[2:]] = df[df.columns[2:]].replace('-', 0)
df[df.columns[2:]] = df[df.columns[2:]].replace(['-', '.'], [0, np.NaN])
df[df.columns[2:]] = df[df.columns[2:]].astype('float')

df.drop(df[df["no_houses_2020"] == 0].index, inplace=True)

## TomTom Search API

To be able to create a map with the neighbourhoods of Amsterdam and levarage on the Foursquare Places API, we will need to geocode the Amsterdam neighbourhoods. With the TomTom Search API we will obtain latitude and longitude for each neighbourhood in Amsterdam. 


###### ! PLEASE NOTE ! 
###### My TomTom key was ommitted from this notebook. Please insert your own TomTom API key below to make the script work. You can get it for free at https://developer.tomtom.com/user/register

In [558]:
tt_key = 'INSERT YOUR KEY HERE'

neighbourhood_tomtom = []
lat = []
lon = []

for nei in df['neighbourhood_name']:
    r = requests.get('https://api.tomtom.com/search/2/structuredGeocode.json?key={}&countryCode=NL&municipality=Amsterdam&municipalitySubdivision={}'.format(tt_key, nei))
    neighbourhood_tomtom.append(r.json()['results'][0]['address']['municipalitySubdivision'])
    lat.append(r.json()['results'][0]['position']['lat'])
    lon.append(r.json()['results'][0]['position']['lon'])
    time.sleep(.050)

df['neighbourhood_tomtom'] = neighbourhood_tomtom
df['lat'] = lat
df['lon'] = lon

r = requests.get('https://api.tomtom.com/search/2/structuredGeocode.json?key={}&countryCode=NL&municipality=Amsterdam'.format(tt_key))
ams_lat = r.json()['results'][0]['position']['lat']
ams_lon = r.json()['results'][0]['position']['lon']

print('Latitude: min = {}, max = {}'.format(df['lat'].min(), df['lat'].max()))
print('Longitude: min = {}, max = {}'.format(df['lon'].min(), df['lon'].max()))
print('Amsterdam: latitude = {}, longitude = {}'.format(ams_lat, ams_lon))

Latitude: min = 52.29549, max = 52.42235
Longitude: min = 4.78065, max = 5.01574
Amsterdam: latitude = 52.37317, longitude = 4.89066


In [559]:
print('\n')
print('Number of distinct neighbourhoods in city of Amsterdam datasets:', len(df['neighbourhood_name'].unique()))
print('Number of distinct neighbourhoods in TomTom Search API:', len(df['neighbourhood_tomtom'].unique()))



Number of distinct neighbourhoods in city of Amsterdam datasets: 98
Number of distinct neighbourhoods in TomTom Search API: 90


#### Drop the city of Amsterdam neighbourhood code and neighbourhood name

In [561]:
df.drop(['neighbourhood_code', 'neighbourhood_name'], axis=1, inplace=True)
print(df.columns)

Index(['no_houses_2016', 'no_houses_2017', 'no_houses_2018', 'no_houses_2019',
       'no_houses_2020', 'no_houses_2020_rooms_1+2', 'no_houses_2020_rooms_5+',
       'no_houses_2020_m2_<40', 'no_houses_2020_m2_40-50',
       'no_houses_2020_m2_50-60', 'no_houses_2020_m2_60-70',
       'no_houses_2020_m2_70-80', 'no_houses_2020_m2_80-90',
       'no_houses_2020_m2_90+', 'no_houses_2020_year_<1946',
       'no_houses_2020_year_1946-1980', 'no_houses_2020_year_1981-1990',
       'no_houses_2020_year_1991-2000', 'no_houses_2020_year_2001-2010',
       'no_houses_2020_year_2010+', 'no_houses_2019_owner_is_resident',
       'no_houses_2019_corporation_rental_total',
       'no_houses_2019_private_rental',
       'avg_real_estate_value_in_eur_2017_per_m2',
       'avg_real_estate_value_in_eur_2018_per_m2',
       'avg_real_estate_value_in_eur_2019_per_m2', 'no_houses_forecasted_2025',
       'no_houses_forecasted_2030', 'no_houses_forecasted_2040',
       'no_houses_forecasted_2050', 'no_resi

#### Reorder columns

In [562]:
column_names = ['neighbourhood_tomtom', 'lat', 'lon', 'no_houses_2016', 'no_houses_2017', 'no_houses_2018', 'no_houses_2019',
           'no_houses_2020', 'no_houses_2020_rooms_1+2', 'no_houses_2020_rooms_5+',
           'no_houses_2020_m2_<40', 'no_houses_2020_m2_40-50',
           'no_houses_2020_m2_50-60', 'no_houses_2020_m2_60-70',
           'no_houses_2020_m2_70-80', 'no_houses_2020_m2_80-90',
           'no_houses_2020_m2_90+', 'no_houses_2020_year_<1946',
           'no_houses_2020_year_1946-1980', 'no_houses_2020_year_1981-1990',
           'no_houses_2020_year_1991-2000', 'no_houses_2020_year_2001-2010',
           'no_houses_2020_year_2010+', 'no_houses_2019_owner_is_resident',
           'no_houses_2019_corporation_rental_total',
           'no_houses_2019_private_rental', 
           'avg_real_estate_value_in_eur_2017_per_m2',
           'avg_real_estate_value_in_eur_2018_per_m2',
           'avg_real_estate_value_in_eur_2019_per_m2', 'no_houses_forecasted_2025',
           'no_houses_forecasted_2030', 'no_houses_forecasted_2040',
           'no_houses_forecasted_2050', 'no_residents_2020_couple_with_children',
           'no_residents_2020_single_parent_family',
           'no_residents_2020_couple_without_children',
           'no_residents_2020_2_singles',
           'no_residents_2020_1_single+single_parent_family',
           'no_residents_2020_living_alone', 'no_residents_2020_3+_singles',
           'no_residents_2020']
df = df.reindex(columns=column_names)

#### Show neighbourhoods named the same in the TomTom Search API

In [563]:
df[df.duplicated(subset='neighbourhood_tomtom', keep=False)].sort_values(by='neighbourhood_tomtom')

,neighbourhood_tomtom,lat,lon,no_houses_2016,no_houses_2017,no_houses_2018,no_houses_2019,no_houses_2020,no_houses_2020_rooms_1+2,no_houses_2020_rooms_5+,no_houses_2020_m2_<40,no_houses_2020_m2_40-50,no_houses_2020_m2_50-60,no_houses_2020_m2_60-70,no_houses_2020_m2_70-80,no_houses_2020_m2_80-90,no_houses_2020_m2_90+,no_houses_2020_year_<1946,no_houses_2020_year_1946-1980,no_houses_2020_year_1981-1990,no_houses_2020_year_1991-2000,no_houses_2020_year_2001-2010,no_houses_2020_year_2010+,no_houses_2019_owner_is_resident,no_houses_2019_corporation_rental_total,no_houses_2019_private_rental,avg_real_estate_value_in_eur_2017_per_m2,avg_real_estate_value_in_eur_2018_per_m2,avg_real_estate_value_in_eur_2019_per_m2,no_houses_forecasted_2025,no_houses_forecasted_2030,no_houses_forecasted_2040,no_houses_forecasted_2050,no_residents_2020_couple_with_children,no_residents_2020_single_parent_family,no_residents_2020_couple_without_children,no_residents_2020_2_singles,no_residents_2020_1_single+single_parent_family,no_residents_2020_living_alone,no_residents_2020_3+_singles,no_residents_2020
93,Bijlmer,52.31680,4.96853,11780.0,11849.0,11858.0,12180.0,12775.0,6362.0,814.0,3270.0,603.0,1484.0,2002.0,967.0,1857.0,2577.0,33.0,2705.0,5518.0,396.0,1207.0,2916.0,2486.0,6928.0,2766.0,2104.0,2299.0,2551.0,14495.0,14662.0,15089.0,16260.0,703.0,1686.0,455.0,1135.0,535.0,6577.0,534.0,12260.0
94,Bijlmer,52.31680,4.96853,12084.0,12671.0,12751.0,13004.0,13002.0,3460.0,2499.0,640.0,349.0,1349.0,1548.0,1450.0,1663.0,5976.0,4.0,6155.0,1327.0,1110.0,2781.0,1625.0,4261.0,6862.0,1881.0,1818.0,2112.0,2444.0,13647.0,14776.0,14962.0,16737.0,1115.0,1884.0,897.0,1127.0,827.0,5145.0,687.0,12631.0
60,Buitenveldert,52.33029,4.87755,7443.0,7499.0,7503.0,7512.0,7528.0,1496.0,1246.0,456.0,569.0,277.0,1465.0,1640.0,990.0,2095.0,181.0,5928.0,228.0,451.0,531.0,209.0,3110.0,1879.0,2523.0,3365.0,4092.0,4509.0,7813.0,8334.0,8515.0,8981.0,836.0,393.0,887.0,843.0,288.0,3354.0,240.0,7003.0
61,Buitenveldert,52.33029,4.87755,4505.0,4502.0,4503.0,4552.0,4555.0,805.0,393.0,572.0,123.0,147.0,963.0,846.0,402.0,1487.0,18.0,2776.0,1237.0,0.0,100.0,424.0,1506.0,1272.0,1774.0,3226.0,3939.0,4531.0,4576.0,4587.0,4587.0,5053.0,451.0,265.0,537.0,484.0,123.0,2265.0,105.0,4299.0
70,IJburg,52.36074,4.98479,5479.0,5504.0,5624.0,5975.0,6264.0,936.0,1532.0,254.0,182.0,138.0,147.0,444.0,1066.0,4031.0,0.0,0.0,0.0,0.0,5153.0,1111.0,2897.0,1924.0,1154.0,3131.0,3462.0,3830.0,6746.0,6639.0,6739.0,6739.0,1394.0,768.0,533.0,716.0,656.0,1685.0,84.0,6049.0
72,IJburg,52.36074,4.98479,2653.0,2799.0,2974.0,3080.0,3081.0,262.0,908.0,110.0,56.0,43.0,24.0,248.0,719.0,1881.0,0.0,0.0,0.0,0.0,2245.0,836.0,1361.0,1015.0,704.0,3238.0,3627.0,3874.0,3359.0,3359.0,3359.0,3359.0,770.0,405.0,231.0,273.0,360.0,804.0,27.0,3006.0
66,Indische Buurt,52.36225,4.94574,6682.0,6648.0,6536.0,6678.0,6704.0,2764.0,273.0,516.0,1302.0,1494.0,924.0,930.0,704.0,810.0,2709.0,179.0,1679.0,559.0,833.0,745.0,1826.0,3613.0,1239.0,4075.0,4764.0,5231.0,6843.0,6845.0,6845.0,6845.0,656.0,485.0,422.0,1177.0,280.0,2948.0,216.0,6346.0
67,Indische Buurt,52.36225,4.94574,4982.0,5030.0,5081.0,5081.0,4962.0,1777.0,196.0,248.0,907.0,737.0,659.0,862.0,784.0,763.0,987.0,290.0,1703.0,1050.0,488.0,444.0,1106.0,3377.0,598.0,3738.0,4573.0,4906.0,5321.0,5321.0,5321.0,5321.0,572.0,433.0,413.0,665.0,203.0,2207.0,146.0,4784.0
37,Osdorp,52.35638,4.80086,8211.0,7997.0,7919.0,7859.0,7928.0,1338.0,794.0,280.0,334.0,1352.0,1949.0,1265.0,974.0,1744.0,0.0,4767.0,861.0,699.0,1177.0,424.0,2212.0,4289.0,1358.0,2345.0,2660.0,2996.0,9104.0,10045.0,10158.0,10543.0,1034.0,677.0,1012.0,677.0,277.0,3354.0,240.0,7636.0
38,Osdorp,52.35638,4.80086,6614.0,6614.0,6615.0,6615.0,6616.0,710.0,843.0,169.0,160.0,654.0,1312.0,909.0,1082.0,2304.0,1.0,1776.0,409.0,541.0,3429.0,460.0,1848.0,3833.0,934.0,2324.0,2621.0,2978.0,6665.0,6855.0,7031.0,7121.0,1208.0,646.0,646.0,575.0,281.0,2168.0,244.0,6223.0


#### Prepare dataframes to merge duplicate neighbourhoods

In [564]:
df_dupl_first = df[df.duplicated(subset='neighbourhood_tomtom', keep='last')].sort_values(by='neighbourhood_tomtom')
df_dupl_first.reset_index(drop=True, inplace=True)
df_dupl_last = df[df.duplicated(subset='neighbourhood_tomtom', keep='first')].sort_values(by='neighbourhood_tomtom')
df_dupl_last.reset_index(drop=True, inplace=True)
columns_sum = ['no_houses_2016', 'no_houses_2017', 'no_houses_2018', 'no_houses_2019',
           'no_houses_2020', 'no_houses_2020_rooms_1+2', 'no_houses_2020_rooms_5+',
           'no_houses_2020_m2_<40', 'no_houses_2020_m2_40-50',
           'no_houses_2020_m2_50-60', 'no_houses_2020_m2_60-70',
           'no_houses_2020_m2_70-80', 'no_houses_2020_m2_80-90',
           'no_houses_2020_m2_90+', 'no_houses_2020_year_<1946',
           'no_houses_2020_year_1946-1980', 'no_houses_2020_year_1981-1990',
           'no_houses_2020_year_1991-2000', 'no_houses_2020_year_2001-2010',
           'no_houses_2020_year_2010+', 'no_houses_2019_owner_is_resident',
           'no_houses_2019_corporation_rental_total',
           'no_houses_2019_private_rental', 'no_houses_forecasted_2025',
           'no_houses_forecasted_2030', 'no_houses_forecasted_2040',
           'no_houses_forecasted_2050', 'no_residents_2020_couple_with_children',
           'no_residents_2020_single_parent_family',
           'no_residents_2020_couple_without_children',
           'no_residents_2020_2_singles',
           'no_residents_2020_1_single+single_parent_family',
           'no_residents_2020_living_alone', 'no_residents_2020_3+_singles',
           'no_residents_2020']
columns_same = ['neighbourhood_tomtom', 'lat', 'lon']
columns_remain = set(column_names) - set(columns_sum) - set(columns_same  )

#### Combine neighbourhoods that were duplicated

In [565]:
df_combined = df_dupl_first[column_names]
df_combined[columns_sum] = df_dupl_first[columns_sum] + df_dupl_last[columns_sum]
df_combined['avg_real_estate_value_in_eur_2017_per_m2'] = round((df_dupl_first['avg_real_estate_value_in_eur_2017_per_m2'] * df_dupl_first['no_houses_2017'] + df_dupl_last['avg_real_estate_value_in_eur_2017_per_m2'] * df_dupl_last['no_houses_2017']) / df_combined['no_houses_2017'], 0)
df_combined['avg_real_estate_value_in_eur_2018_per_m2'] = round((df_dupl_first['avg_real_estate_value_in_eur_2018_per_m2'] * df_dupl_first['no_houses_2018'] + df_dupl_last['avg_real_estate_value_in_eur_2018_per_m2'] * df_dupl_last['no_houses_2018']) / df_combined['no_houses_2018'], 0)
df_combined['avg_real_estate_value_in_eur_2019_per_m2'] = round((df_dupl_first['avg_real_estate_value_in_eur_2019_per_m2'] * df_dupl_first['no_houses_2019'] + df_dupl_last['avg_real_estate_value_in_eur_2019_per_m2'] * df_dupl_last['no_houses_2019']) / df_combined['no_houses_2019'], 0)

df.drop_duplicates(subset='neighbourhood_tomtom', keep=False, inplace=True)

df = df.append(df_combined, ignore_index=True).sort_values(by='neighbourhood_tomtom')
df.reset_index(drop=True, inplace=True)
df.rename(columns={'neighbourhood_tomtom':'neighbourhood'}, inplace=True)

#### Calculating metrics & remove not needed columns

In [568]:
df['perc_change_no_houses_2016_2020'] = round(((df['no_houses_2020'] - df['no_houses_2016']) / df['no_houses_2016'])*100, 0)
df['perc_houses_2020_rooms_1+2'] = round((df['no_houses_2020_rooms_1+2'] / df['no_houses_2020'])*100, 0)
df['perc_houses_2020_rooms_5+'] = round((df['no_houses_2020_rooms_5+'] / df['no_houses_2020'])*100, 0)
df['perc_houses_2020_m2_<50'] = round(((df['no_houses_2020_m2_<40'] + df['no_houses_2020_m2_40-50']) / df['no_houses_2020'])*100, 0)
df['perc_houses_2020_m2_90+'] = round((df['no_houses_2020_m2_90+'] / df['no_houses_2020'])*100, 0)
df['perc_houses_2020_year_<1946'] = round((df['no_houses_2020_year_<1946'] / df['no_houses_2020'])*100, 0)
df['perc_houses_2020_year_2010+'] = round((df['no_houses_2020_year_2010+'] / df['no_houses_2020'])*100, 0)
df['perc_houses_2019_owner_is_resident'] = round((df['no_houses_2019_owner_is_resident'] / df['no_houses_2019'])*100, 0)
df['perc_houses_2019_private_rental'] = round((df['no_houses_2019_private_rental'] / df['no_houses_2019'])*100, 0)
df['perc_change_avg_real_estate_value_in_eur_2017_2019_per_m2'] = round(((df['avg_real_estate_value_in_eur_2019_per_m2'] - df['avg_real_estate_value_in_eur_2017_per_m2']) / df['avg_real_estate_value_in_eur_2017_per_m2'])*100, 0)
df['perc_change_no_houses_forecasted_2020_2030'] = round(((df['no_houses_forecasted_2030'] - df['no_houses_2020']) / df['no_houses_forecasted_2030'])*100, 0)
df['perc_change_no_houses_forecasted_2020_2050'] = round(((df['no_houses_forecasted_2050'] - df['no_houses_2020']) / df['no_houses_forecasted_2050'])*100, 0)
df['perc_residents_2020_living_alone'] = round((df['no_residents_2020_living_alone'] / df['no_residents_2020'])*100, 0)
df['perc_residents_2020_couple_with_children'] = round((df['no_residents_2020_couple_with_children'] / df['no_residents_2020'])*100, 0)

df = df[['neighbourhood', 'lat', 'lon', 
    'no_houses_2020',
    'perc_change_no_houses_2016_2020',
    'perc_change_no_houses_forecasted_2020_2030',
    'perc_change_no_houses_forecasted_2020_2050',
    'perc_houses_2020_rooms_1+2', 
    'perc_houses_2020_rooms_5+',
    'perc_houses_2020_m2_<50',
    'perc_houses_2020_m2_90+', 
    'perc_houses_2020_year_<1946',
    'perc_houses_2020_year_2010+', 
    'perc_houses_2019_owner_is_resident',
    'perc_houses_2019_private_rental',
    'perc_change_avg_real_estate_value_in_eur_2017_2019_per_m2',
    'perc_residents_2020_living_alone',
    'perc_residents_2020_couple_with_children',
    'avg_real_estate_value_in_eur_2019_per_m2']]

#### Drop rows with any missing values and print descriptive statistics of the dataframe

In [569]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.describe()

,lat,lon,no_houses_2020,perc_change_no_houses_2016_2020,perc_change_no_houses_forecasted_2020_2030,perc_change_no_houses_forecasted_2020_2050,perc_houses_2020_rooms_1+2,perc_houses_2020_rooms_5+,perc_houses_2020_m2_<50,perc_houses_2020_m2_90+,perc_houses_2020_year_<1946,perc_houses_2020_year_2010+,perc_houses_2019_owner_is_resident,perc_houses_2019_private_rental,perc_change_avg_real_estate_value_in_eur_2017_2019_per_m2,perc_residents_2020_living_alone,perc_residents_2020_couple_with_children,avg_real_estate_value_in_eur_2019_per_m2
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,52.363992,4.891957,5071.488636,16.295455,10.090909,16.818182,29.454545,11.261364,20.852273,27.363636,41.625000,15.875000,32.443182,31.568182,28.920455,45.056818,11.181818,5033.034091
std,0.025590,0.050891,3781.859717,52.781736,18.753474,22.865195,17.228624,11.528570,16.024077,19.995506,34.977271,22.292111,15.338973,20.254584,9.571535,11.015515,5.999129,1340.317156
min,52.295490,4.780650,197.000000,-5.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,16.000000,19.000000,0.000000,2478.000000
25%,52.349773,4.861228,2721.500000,0.000000,0.000000,1.000000,15.750000,4.000000,8.000000,13.000000,1.000000,3.000000,22.750000,16.000000,23.750000,39.000000,7.000000,3835.500000
50%,52.364735,4.890150,4616.000000,2.000000,2.000000,7.000000,27.000000,7.000000,19.000000,22.500000,46.000000,8.000000,29.000000,32.000000,27.000000,47.000000,9.000000,5284.500000
75%,52.378552,4.927690,6224.500000,6.000000,9.250000,23.000000,41.250000,12.000000,30.250000,31.500000,72.250000,16.000000,39.000000,43.250000,32.000000,52.000000,15.250000,6013.250000
max,52.422350,5.015740,25777.000000,340.000000,94.000000,97.000000,97.000000,70.000000,94.000000,88.000000,96.000000,97.000000,89.000000,99.000000,84.000000,91.000000,27.000000,7339.000000


## Visualize a map of Amsterdam's neighbourhoods

In [570]:
m_ams = folium.Map(location=[ams_lat, ams_lon], zoom_start=11)

for lat, lon, neighbourhood, no_houses_2020, perc_houses_2020_year_before_1946 in zip(df['lat'], df['lon'], df['neighbourhood'], df['no_houses_2020'], df['perc_houses_2020_year_<1946']):
    folium.CircleMarker([lat, lon], 
                        tooltip=neighbourhood,
                        radius=8,
                        fill=True,
                        color='#3279a8',
                        fill_color='#3279a8',
                        fill_opacity=0.5,
                        weight=2,
                        popup='<b><nobr>{}</nobr></b><br /><nobr>No. houses in 2020: <b>{}</b></nobr><br /><nobr>Percentage of houses built before 1946: <b>{}%</b></nobr>'.format(neighbourhood, int(no_houses_2020), int(perc_houses_2020_year_before_1946)),
                        ).add_to(m_ams)

m_ams

## Foursquare Places API

The Foursquare Places API will be used to gain insight into the number and type of venues in each of the neighbourhoods.

###### ! PLEASE NOTE !
###### Remember to insert you client ID and client secret to make the script work!

#### Set client id, secret,  limit and radius

In [571]:
client_id = 'INSERT YOUR CLIENT ID'
client_secret = 'INSERT YOUR CLIENT SECRET'
version = '20200429'
limit = 1000
radius_meter = 500

#### Retrieve venues in a proximity of 500m from the neighbourhoods' latitude and longitude

In [572]:
no_venues = []
venues = []

for lat, lon, neighbourhood in zip(df['lat'], df['lon'], df['neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&limit={}".format(
        client_id,
        client_secret, 
        version,
        lat,
        lon,
        limit)
    response = requests.get(url)
    ven = response.json()['response']['venues']
    no_venues.append(len(ven))
    venues.append(ven)
    
df['no_venues'] = no_venues
df['venues_list'] = venues

#### Create a dataframe with all categories of venues retrieved

In [573]:
df_venues = pd.DataFrame()
venue_names = []
venue_categories = []

for venues_list in df['venues_list']:
    for venue in venues_list:
        venue_names.append(venue['name'])
        try:
            venue_categories.append(venue['categories'][0]['name'])
        except:
            venue_categories.append(np.nan)

df_venues['name'] = venue_names
df_venues['category'] = venue_categories

#### Create a list of top 20 categories in Amsterdam & count for each neighbourhood the number of top 20 categories

In [574]:
df_venues_grouped = df_venues.groupby(by='category').count().sort_values(by='name', ascending=False)
cat_sort = list(df_venues_grouped.index)
top20_cat = cat_sort[0:20]

for category in top20_cat:
    df[category] = 0
    
for column_name in top20_cat:
    for index in range(0, df.shape[0]):
        for venue in df.loc[index, 'venues_list']:
            try:
                if venue['categories'][0]['name'] == column_name:
                    df.loc[index, column_name] += 1
            except:
                df.loc[index, column_name] += 0

df.drop(columns='venues_list', inplace=True)

for column in top20_cat:
    df[column] = round(100*(df[column]/df['no_venues']), 0)

### Cluster Amsterdam Neighbourhoods

We will use k-means clustering as we want to explore the neighbourhoods and don't have a defined outcome. 

In [576]:
# set number of clusters
k = 5

fit_data = df.drop('neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(fit_data)

# add clusters to dataframe
df['cluster'] = kmeans.labels_

# 4. Results

### Visualize Clusters

In [577]:
m_ams = folium.Map(location=[ams_lat, ams_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, neighbourhood, no_houses_2020, perc_houses_2020_year_before_1946, cluster in zip(df['lat'], df['lon'], df['neighbourhood'], df['no_houses_2020'], df['perc_houses_2020_year_<1946'], df['cluster']):
    folium.CircleMarker([lat, lon], 
                        tooltip=neighbourhood,
                        radius=8,
                        fill=True,
                        color=rainbow[cluster],
                        fill_color=rainbow[cluster],
                        fill_opacity=0.5,
                        weight=2,
                        popup='<b><nobr>{}</nobr></b><br /><nobr>No. houses in 2020: <b>{}</b></nobr><br /><nobr>Percentage of houses built before 1946: <b>{}%</b></nobr><br /><nobr>Cluster: <b>{}</b></nobr><br /><nobr>'.format(neighbourhood, int(no_houses_2020), int(perc_houses_2020_year_before_1946), cluster),
                        ).add_to(m_ams)

m_ams

### Describe Clusters

#### Number of neighbourhoods in each cluster

In [578]:
df[['neighbourhood', 'cluster']].groupby('cluster').count()

,neighbourhood
cluster,
0,31
1,30
2,9
3,1
4,17


#### Average metrics for each cluster

In [579]:
df.groupby('cluster').mean().astype('int')

,lat,lon,no_houses_2020,perc_change_no_houses_2016_2020,perc_change_no_houses_forecasted_2020_2030,perc_change_no_houses_forecasted_2020_2050,perc_houses_2020_rooms_1+2,perc_houses_2020_rooms_5+,perc_houses_2020_m2_<50,perc_houses_2020_m2_90+,perc_houses_2020_year_<1946,perc_houses_2020_year_2010+,perc_houses_2019_owner_is_resident,perc_houses_2019_private_rental,perc_change_avg_real_estate_value_in_eur_2017_2019_per_m2,perc_residents_2020_living_alone,perc_residents_2020_couple_with_children,avg_real_estate_value_in_eur_2019_per_m2,no_venues,Office,Residential Building (Apartment / Condo),Building,Salon / Barbershop,Coworking Space,Bus Stop,Bar,Art Gallery,Playground,Restaurant,Doctor's Office,Bakery,Dentist's Office,Café,Medical Center,Tech Startup,Event Space,Coffee Shop,Hotel,Park
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,52,4,6290,5,7,13,28,10,19,24,36,12,28,26,29,45,11,4777,105,7,3,2,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0
1,52,4,3421,27,9,13,35,10,24,24,60,19,29,43,29,48,8,6034,109,9,2,2,1,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,0
2,52,4,11249,3,6,8,29,9,20,26,26,9,31,27,26,44,12,4731,102,5,3,2,2,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1
3,52,4,25777,8,12,22,38,13,19,33,0,18,27,18,28,47,7,2496,90,2,4,8,2,2,4,1,1,0,0,6,1,3,0,1,0,1,0,0,1
4,52,4,1272,23,18,31,20,15,17,38,28,18,44,23,28,39,15,4041,112,10,1,3,0,1,2,0,0,1,0,0,0,1,1,0,0,1,0,0,1


# 5. Discussion

Cluster 3 is an outlier with just one neighbourhood that didn't fit in any other cluster. 

Cluster 0 and 4 are rather small neighbourhoods that have grown substantially in number of houses in the last 4 years. 

Especially neighbourhoods in cluster 0 are expected to grow further in the next decades. Houses in these neighbourhoods are rather new and spacious. 

The venue data doesn't seem to be very helpful for the clustering here. Foursquare doesn't seem to deliver a great deal of accuracy for the city of Amsterdam. Many venues were lacking a category and most restaurants, bars etc. were not returned by the Foursquare API.  